# Tarea para el Hogar 04

##  1. Big Picture

LightGBM es el algoritmo estado del arte para datasets estructurados.
<br> La Bayesian Optimization es el estado del arte para optimización de hiperparámetros
<br> Las soluciones a las tres competencias de la asignatura contendrán LightGBMs y Bayesian Optimizations
<br> LightGBM ha aumentado en forma no darwiniana sus hiperparámetros en los últimos ocho años; no todos los existentes son útiles.
<br> Es necesario lograr entender cuales son los hiperparámetros relevantes de LightGBM

## 2. Hiperparámetros del LightGBM

Los objetivos de esta tarea son:


*   Aumentar la rentabilidad de la campaña de marketing de retención proactiva de clientes.
*   Generar un mejor modelo optimizando sus hiperparámetros
*   Conceptual : investigar los mas relevantes hiperparámetros de LightGBM
*   Familiarizarse con la Bayesian Optimization, sus largos tiempos de corrida y opciones para reducirlos
*   Familiarizarse con el uso de máquinas virtuales de Google Colab
*   Ver un pipeline completo de optimización de hiperparámetros y puesta en producción

LightGBM cuenta con mas de 60 hiperparámetros, siendo posible utilizar 40 al mismo tiempo, aunque no razonable.
<br> La documentación oficial de los hiperparámetros de LightGBM es  https://lightgbm.readthedocs.io/en/latest/Parameters.html#core-parameters


Se lo alerta sobre que una Optimizacion Bayesiana lleva varias horas de corrida, y usted deberá correr VARIAS optimizaciones para descubrir cuales parámetros conviene optimizar.
<br> A pesar que la próxima clase es recien en viernes 01 de agosto, inicie la tarea con tiempo, aprenda a planificar estratégicamente sus corridas como un@ científ@  de datos.

Es necesario investigar cuales son los hiperparámetros de LightGBM que vale la pena optimizar en una Bayesian Optimization, ya que los realmente utiles son apenas un reducido subconjunto.
<br>Usted deberá investigar cuales son los hiperparámetros mas relevantes de LightGBM, su primer alternativa es preguntándole a su amigo con capacidades especiales ChatGPT o sus endogámicos familiares Claude, DeepSeek, Gemini, Grok, etc
<br> La segunda alternativa es la propia documentación de LightGBM  https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html


Adicionalmente podra buscar información como la que proveen esta diminuta muestra aleatoria de artículos ligeros:
*  https://medium.com/@sarahzouinina/a-deep-dive-into-lightgbm-how-to-choose-and-tune-parameters-7c584945842e
*  https://www.kaggle.com/code/somang1418/tuning-hyperparameters-under-10-minutes-lgbm
*  https://towardsdatascience.com/beginners-guide-to-the-must-know-lightgbm-hyperparameters-a0005a812702/


<br>  La muestra anterior se brinda a modo de ejemplo, usted deberá buscar muuuuchas  fuentes adicionales de información
<br> Tenga presente que LightGBM es el estado del arte en modelado predictivo para datasets estructurado, que son el 90% del trabajo del 95% de los Data Scientists en Argentina.

El desafío de esta tarea es:
* Qué hiperparparámetros conviene optimizar?  Las recomendaciones de los artículos ligeros es siempre sensata?  Sus autores realmente hicieron experimentos o son siemplemente escritores de entretenimiento carente de base científica?
* Elegidos los hiperparámetros, cual es el  <desde, hasta> que se debe utilizar en la Bayesian Optimization ?
* Realmente vale la pena optimizar 10 o 16 hiperparámetros al mismo tiempo ?  No resulta contraproducente una búsqueda en un espacio de tal alta dimensionalidad ?

#### 2.1  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



## Generacion de la clase_ternaria

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

In [29]:
cat("Comienzo de Corrida: ", format(Sys.time(), "%a %b %d %X %Y"), "\n")

Comienzo de Corrida:  jue. oct. 09 23:55:16 2025 


In [30]:
if( !require("data.table")) install.packages("data.table")
require( "data.table" )

setwd("/Users/sergiovolta/proyectos/eyf/")

# leo el dataset
dataset <- fread("content/datasets/competencia_01_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

fwrite( dataset,
    file =  "content/datasets/competencia_01.csv.gz",
    sep = ","
)

In [31]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
202101,BAJA+1,622
202101,BAJA+2,825
202101,CONTINUA,160080
202102,BAJA+1,831
202102,BAJA+2,1032
202102,CONTINUA,160292
202103,BAJA+1,1039
202103,BAJA+2,951
202103,CONTINUA,161119


### 2.2 Optimizacion Hiperparámetros

Esta parte se debe correr con el runtime en lenguaje R Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

### 2.2.1 Inicio

limpio el ambiente de R

In [32]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "jue. oct. 09 23:55:23 2025"

In [33]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),limit (Mb),max used,(Mb)
Ncells,2387495,127.6,4881178,260.7,NA,4881178,260.7
Vcells,4269561,32.6,594329258,4534.4,24576,635450621,4848.2


### 2.2.2 Carga de Librerias

Esta parte lleva varios minutos la primera vez en Google Colab

In [34]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

In [ ]:
PARAM <- list()
PARAM$experimento <- 6021
PARAM$semilla_primigenia <- 611033


In [36]:
# training y future
PARAM$train <- c(202102, 202103, 202104)
PARAM$train_final <- c(202101, 202102, 202103, 202104)
PARAM$future <- c(202106)
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

In [37]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.10

In [38]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000, 
  num_threads = parallel::detectCores() - 1 
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [39]:
# Aqui se cargan los bordes de los hiperparametros de la BO
PARAM$hypeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_iterations", lower= 8L, upper= 2048L),
  makeNumericParam("learning_rate", lower= 0.01, upper= 0.3),
  makeNumericParam("feature_fraction", lower= 0.1, upper= 1.0),
  makeIntegerParam("num_leaves", lower= 8L, upper= 2048L),
  makeIntegerParam("min_data_in_leaf", lower= 1L, upper= 8000L)
)

A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

In [40]:
PARAM$hyperparametertuning$iteraciones <- 25 # iteraciones bayesianas

In [41]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [42]:
# inicializo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [43]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

### 2.2.4  Preprocesamiento

In [44]:
getwd()
#

[1] "/Users/sergiovolta/proyectos/eyf"

In [45]:
# carpeta de trabajo

setwd("/Users/sergiovolta/proyectos/eyf/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/Users/sergiovolta/proyectos/eyf/content/buckets/b1/exp/", experimento_folder ))

In [46]:
# lectura del dataset
dataset <- fread("/Users/sergiovolta/proyectos/eyf/content/datasets/competencia_01.csv.gz", 
stringsAsFactors= TRUE)

cat("Número de columnas en dataset con claseternaria: ", ncol(dataset), "\n") 
cat("Número de filas en dataset con claseternaria: ", nrow(dataset), "\n")

Número de columnas en dataset con claseternaria:  155 
Número de filas en dataset con claseternaria:  978439 


In [47]:
dataset_train <- dataset[foto_mes %in% PARAM$train]

In [48]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [49]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

# set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# dataset_train[, azar := runif(nrow(dataset_train))]
# dataset_train[, training := 0L]

# dataset_train[
#   foto_mes %in%  PARAM$train &
#     (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
#   training := 1L
# ]

# Lo anterior fue reemplazado por lo siguiente que respeta el numero de cliente
# Es decir, si un cliente es seleccionado, se muestran todas sus filas

# Fijar semilla para reproducibilidad
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")

# Paso 1: Crear tabla con un registro por cliente
clientes <- dataset_train[
  foto_mes %in% PARAM$train,
  .(tiene_baja = any(clase_ternaria %in% c("BAJA+1", "BAJA+2"))),
  by = numero_de_cliente
]

# Paso 2: Decidir si se incluye el cliente en el set de entrenamiento
clientes[, incluir := fifelse(
  tiene_baja,
  TRUE,
  runif(.N) <= PARAM$trainingstrategy$undersampling
)]

# Paso 3: Inicializar la columna 'training' en el dataset original
dataset_train[, training := 0L]

# Paso 4: Marcar como training las filas de clientes seleccionados, solo en meses de entrenamiento
dataset_train[
  numero_de_cliente %in% clientes[incluir == TRUE, numero_de_cliente] &
    foto_mes %in% PARAM$train,
  training := 1L
]

cat("Número de filas de clientes seleccionados para training: ", nrow(dataset_train[training == 1]), "\n")

Número de filas de clientes seleccionados para training:  57063 


# Feature Engineering: Cuenta de valores Nulos por fila

In [50]:
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria", "clase01", "azar", "training")
cols_a_contar <- setdiff(names(dataset_train), cols_excluir)
dataset_train[, cantidad_na := rowSums(is.na(.SD)), .SDcols = cols_a_contar]


# Cálculo de Lags

In [51]:
cat("Número de columnas antes del lag:", ncol(dataset_train), "\n")

# Lags
# Lista de columnas a excluir del cálculo de lags
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria", "clase01", "azar", "training")
# Determinar columnas a las que sí se les aplica el lag
cols_usar <- setdiff(names(dataset_train), cols_excluir)

# Ordenar para que el lag tenga sentido temporal
setorder(dataset_train, numero_de_cliente, foto_mes)

# Para cada columna a laggear, crear lag01_ y deltalag01_
for (col in cols_usar) {
  
  # Nombre de las nuevas columnas
  col_lag1 <- paste0("lag01_", col)
  col_delta1 <- paste0("deltalag01_", col)
  col_lag2 <- paste0("lag02_", col)
  col_delta2 <- paste0("deltalag02_", col)
  
  # Generar el lag por cliente
  dataset_train[, (col_lag1) := shift(.SD[[1]], n = 1, type = "lag"), by = numero_de_cliente, .SDcols = col]
  dataset_train[, (col_lag2) := shift(.SD[[1]], n = 2, type = "lag"), by = numero_de_cliente, .SDcols = col]
  
  # Calcular la diferencia con el valor actual
  dataset_train[, (col_delta1) := get(col) - get(col_lag1)]
  dataset_train[, (col_delta2) := get(col) - get(col_lag2)]

  # Quito los lags y me quedo con los deltas para economizar
  dataset_train[, c(col_lag1, col_lag2) := NULL]

}

cat("Número de columnas después del lag:", ncol(dataset_train), "\n")
cat("Número de filas después del lag:", nrow(dataset_train), "\n")

Número de columnas antes del lag: 158 
Número de columnas después del lag: 464 
Número de filas después del lag: 488682 


# Normalización de Variables Numéricas

In [ ]:

# columnas que NO voy a modificar
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria", "clase01", "azar", "training")

# obtener columnas numéricas que NO están en la lista de exclusión
cols_normalizar <- setdiff(
  names(dataset_train)[sapply(dataset_train, is.numeric)],
  cols_excluir
)

# función de normalización personalizada
# Esta función rankea los valores de cada columna dentro del valor de 
# foto_mes correspondiente al de su fila
normalizar_custom_por_foto_mes <- function(dt, columnas) {
  for (col in columnas) {
    dt[, (col) := {
      x <- get(col)
      x_norm <- x  # preserva NA

      idx_pos <- which(!is.na(x) & x > 0)
      idx_neg <- which(!is.na(x) & x < 0)

      if (length(idx_pos) > 0) {
        max_pos <- max(x[idx_pos], na.rm = TRUE)
        if (!is.infinite(max_pos) && max_pos != 0) {
          x_norm[idx_pos] <- x[idx_pos] / max_pos
        }
      }

      if (length(idx_neg) > 0) {
        min_neg <- min(x[idx_neg], na.rm = TRUE)
        if (!is.infinite(min_neg) && min_neg != 0) {
          x_norm[idx_neg] <- -x[idx_neg] / abs(min_neg)
          x_norm[idx_neg] <- -x_norm[idx_neg]
        }
      }

      x_norm
    }, by = foto_mes]
  }
}


# aplicar la función a las columnas seleccionadas
normalizar_custom_por_foto_mes(dataset_train, cols_normalizar)


cat("Número de Columnas después de normalizar: ", ncol(dataset_train))

Número de Columnas después de normalizar:  464

In [53]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [54]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

[1] 57063

[1] 461

2.2.5 Configuracion Bayesian Optimization

In [55]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [56]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hypeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


2.2.6 Corrida Bayesian Optimization

In [57]:
cat("Inicio de la corrida Bayesiana: ", format(Sys.time(), "%a %b %d %X %Y"), "\n")

Inicio de la corrida Bayesiana:  vie. oct. 10 00:01:15 2025 


In [58]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
    cat("No encontré una corrida anterior. Iniciando desde cero...")
    bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
    cat("Encontré una corrida anterior. Continúo con el proceso interrumpido...")
    bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

No encontré una corrida anterior. Iniciando desde cero...

Computing y column(s) for design. Not provided.

vie. oct. 10 00:01:33 2025 AUC 0.911676683215186

vie. oct. 10 00:03:45 2025 AUC 0.909416284101439

vie. oct. 10 00:06:42 2025 AUC 0.91113639760057

vie. oct. 10 00:07:15 2025 AUC 0.906697454526783

vie. oct. 10 00:07:44 2025 AUC 0.908402319339392

vie. oct. 10 00:09:00 2025 AUC 0.903910757113841

vie. oct. 10 00:09:21 2025 AUC 0.901154799340117

vie. oct. 10 00:14:03 2025 AUC 0.913212123126063

vie. oct. 10 00:15:18 2025 AUC 0.90936152306388

vie. oct. 10 00:19:57 2025 AUC 0.906805369691839

vie. oct. 10 00:20:46 2025 AUC 0.91279741718944

vie. oct. 10 00:23:47 2025 AUC 0.905445065034363

vie. oct. 10 00:24:27 2025 AUC 0.915333422681903

vie. oct. 10 00:26:15 2025 AUC 0.903187569112169

vie. oct. 10 00:28:36 2025 AUC 0.917501964344411

vie. oct. 10 00:28:38 2025 AUC 0.87775343098189

vie. oct. 10 00:31:23 2025 AUC 0.910983309625493

vie. oct. 10 00:32:31 2025 AUC 0.902862461449911

vie. oct. 10 00:34:25 2025 AUC 0.897823119553181

vie. 

In [59]:
cat("Final de la corrida Bayesiana: ", format(Sys.time(), "%a %b %d %X %Y"), "\n")

Final de la corrida Bayesiana:  vie. oct. 10 02:21:54 2025 


In [60]:

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "num_iterations"   "learning_rate"    "feature_fraction" "num_leaves"      
 [5] "min_data_in_leaf" "y"                "dob"              "eol"             
 [9] "error.message"    "exec.time"        "ei"               "error.model"     
[13] "train.time"       "prop.type"        "propose.time"     "se"              
[17] "mean"

In [61]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [62]:
write_yaml( PARAM, file="PARAM.yml")

In [63]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   num_iterations learning_rate feature_fraction num_leaves min_data_in_leaf
            <int>         <num>            <num>      <int>            <int>
1:           2048    0.01000379        0.6927896        756                3
[1] 0.9261518


## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [70]:
setwd("/Users/sergiovolta/proyectos/eyf/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/Users/sergiovolta/proyectos/eyf/content/buckets/b1/exp/", experimento ))

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la  optimización bayesiana

In [76]:
# lectura del dataset
dataset <- fread("/Users/sergiovolta/proyectos/eyf/content/datasets/competencia_01.csv.gz", stringsAsFactors= TRUE)
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [77]:
dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

clase_ternaria,N
<fct>,<int>
CONTINUA,642824
BAJA+2,3938
BAJA+1,3447


In [ ]:
# Vuelvo a agregar los lags
# Lags
# Lista de columnas a excluir del cálculo de lags
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria", "clase01", "azar", "training")
cols_usar <- setdiff(names(dataset_train), cols_excluir)

# Contar nulos
dataset_train[, cantidad_na := rowSums(is.na(.SD)), .SDcols = cols_usar]

# Determinar columnas a las que sí se les aplica el lag
cols_usar <- setdiff(names(dataset_train), cols_excluir)

# Ordenar para que el lag tenga sentido temporal
setorder(dataset_train, numero_de_cliente, foto_mes)

# Para cada columna a laggear, crear lag01_ y deltalag01_
for (col in cols_usar) {
  
  # Nombre de las nuevas columnas
  # Nombre de las nuevas columnas
  col_lag1 <- paste0("lag01_", col)
  col_delta1 <- paste0("deltalag01_", col)
  col_lag2 <- paste0("lag02_", col)
  col_delta2 <- paste0("deltalag02_", col)

  
  # Generar el lag por cliente
  dataset_train[, (col_lag1) := shift(.SD[[1]], n = 1, type = "lag"), by = numero_de_cliente, .SDcols = col]
  dataset_train[, (col_lag2) := shift(.SD[[1]], n = 2, type = "lag"), by = numero_de_cliente, .SDcols = col]
  
  # Calcular la diferencia con el valor actual
  dataset_train[, (col_delta1) := get(col) - get(col_lag1)]
  dataset_train[, (col_delta2) := get(col) - get(col_lag2)]

  # Quito los lags y me quedo sólo con los deltas
  dataset_train[, c(col_lag1, col_lag2) := NULL]
}

# Normalizo

# obtener columnas numéricas que NO están en la lista de exclusión
cols_normalizar <- setdiff(
  names(dataset_train)[sapply(dataset_train, is.numeric)],
  cols_excluir
)

# La función de normalización personalizada ya fue definida
# aplicar la función a las columnas seleccionadas
normalizar_custom_por_foto_mes(dataset_train, cols_normalizar)
cat("Número de columnas en dataset_train: ", ncol(dataset_train), "\n")




head(dataset_train, 20)


Número de columnas en dataset_train:  463 


numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,deltalag01_Visa_mconsumototal,deltalag02_Visa_mconsumototal,deltalag01_Visa_cconsumos,deltalag02_Visa_cconsumos,deltalag01_Visa_cadelantosefectivo,deltalag02_Visa_cadelantosefectivo,deltalag01_Visa_mpagominimo,deltalag02_Visa_mpagominimo,deltalag01_cantidad_na,deltalag02_cantidad_na
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
249221323,202101,1,0,0,0.4554455,0.24734043,0.0021581895,0.003138512,0.0022540183,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
249221323,202102,1,0,0,0.4554455,0.25000000,0.0023800439,0.003821088,0.0021700025,⋯,4.864920e-04,NA,0.016949153,NA,0,NA,-9.698015e-04,NA,0.0000000,NA
249221323,202103,1,0,0,0.4554455,0.25265957,0.0037545203,0.004804490,0.0013329837,⋯,1.495567e-03,0.0019858982,-0.008196721,0.01086957,0,0,-3.349260e-03,-4.318261e-03,0.0000000,0.0000000
249221323,202104,1,0,0,0.4554455,0.25531915,0.0037653737,0.005838506,0.0025927797,⋯,-1.290218e-03,0.0001903185,-0.008196721,-0.01282051,0,0,0.000000e+00,-3.348639e-03,0.0000000,0.0000000
249227600,202101,1,0,0,0.4158416,0.73936170,0.0003488050,0.003064646,0.0007388770,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
249227600,202102,1,0,0,0.4158416,0.74202128,0.0005088571,0.002930977,0.0004846604,⋯,1.265251e-03,NA,0.008474576,NA,0,NA,1.337499e-04,NA,0.0000000,NA
249227600,202103,1,0,0,0.4158416,0.74468085,0.0005539154,0.002760473,0.0008049927,⋯,-2.494721e-03,-0.0012482414,-0.032786885,-0.01923077,0,0,-5.829408e-05,7.317651e-05,0.0000000,0.0000000
249227600,202104,1,0,0,0.4158416,0.74734043,0.0007825602,0.002824591,0.0013170940,⋯,1.307198e-03,-0.0012066603,0.042372881,0.01086957,0,0,2.866069e-04,2.109205e-04,0.0000000,0.0000000
249234235,202101,1,0,0,0.5346535,0.04521277,0.0005669620,0.002110157,0.0003073172,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [79]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

In [80]:
# carpeta de trabajo

setwd("/Users/sergiovolta/proyectos/eyf/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/Users/sergiovolta/proyectos/eyf/content/buckets/b1/exp/", experimento_folder ))


#### Final Training Hyperparameters

In [81]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 611033

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$max_bin
[1] 31

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 2048

$learning_rate
[1] 0.01000379

$feature_fraction
[1] 0.6927896

$num_leaves
[1] 756

$min_data_in_leaf
[1] 3

$num_threads
[1] 9

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria y mucho menos cross validation.

In [82]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [83]:
# entreno LightGBM

modelo_final <- lgb.train(
  data= dtrain_final,
  param= param_normalizado
)

In [84]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(modelo_final))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [85]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...
lgb.save(modelo_final, "modelo.txt" )

### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% c(202104, 202105, 202106)]

# Vuelvo a agregar los lags
# Lags
# Lista de columnas a excluir del cálculo de lags
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria", "clase01", "azar", "training")
cols_usar <- setdiff(names(dfuture), cols_excluir)

dfuture[, cantidad_na := rowSums(is.na(.SD)), .SDcols = cols_usar]

# Determinar columnas a las que sí se les aplica el lag
cols_usar <- setdiff(names(dfuture), cols_excluir)

# Ordenar para que el lag tenga sentido temporal
setorder(dfuture, numero_de_cliente, foto_mes)

# Para cada columna a laggear, crear lag01_ y deltalag01_
for (col in cols_usar) {
  
  # Nombre de las nuevas columnas
  col_lag1 <- paste0("lag01_", col)
  col_delta1 <- paste0("deltalag01_", col)
  col_lag2 <- paste0("lag02_", col)
  col_delta2 <- paste0("deltalag02_", col)

  
# Generar el lag por cliente
  dfuture[, (col_lag1) := shift(.SD[[1]], n = 1, type = "lag"), by = numero_de_cliente, .SDcols = col]
  dfuture[, (col_lag2) := shift(.SD[[1]], n = 2, type = "lag"), by = numero_de_cliente, .SDcols = col]
  
  # Calcular la diferencia con el valor actual
  dfuture[, (col_delta1) := get(col) - get(col_lag1)]
  dfuture[, (col_delta2) := get(col) - get(col_lag2)]

  # Quedarme sólo con los deltas
  dfuture[, c(col_lag1, col_lag2) := NULL]
}

# Normalizo

# columnas que NO querés modificar
cols_excluir <- c("numero_de_cliente", "foto_mes", "clase_ternaria", "clase01", "azar", "training")

# obtener columnas numéricas que NO están en la lista de exclusión
cols_normalizar <- setdiff(
  names(dfuture)[sapply(dfuture, is.numeric)],
  cols_excluir
)

# La función de normalización personalizada ya fue definida
# aplicar la función a las columnas seleccionadas

normalizar_custom_por_foto_mes(dfuture, cols_normalizar)  
cat("Número de columnas en dfuture: ", ncol(dfuture), "\n")


dfuture <- dfuture[foto_mes %in% PARAM$future]
head(dfuture, 20)



# aplico el modelo a los datos nuevos
prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

Número de columnas en dfuture:  463 


numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,deltalag01_Visa_mconsumototal,deltalag02_Visa_mconsumototal,deltalag01_Visa_cconsumos,deltalag02_Visa_cconsumos,deltalag01_Visa_cadelantosefectivo,deltalag02_Visa_cadelantosefectivo,deltalag01_Visa_mpagominimo,deltalag02_Visa_mpagominimo,deltalag01_cantidad_na,deltalag02_cantidad_na
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
249221323,202106,1,0,0,0.4646465,0.25925926,0.0028027513,0.005737140,0.0029711871,⋯,2.013480e-03,0.0036195684,0.006060606,0.006134969,0,0,2.063709e-04,1.614088e-03,0.00000000,0.00000000
249227600,202106,1,0,0,0.4242424,0.74867725,0.0015918750,0.002444510,0.0019620289,⋯,-2.629248e-04,-0.0011474122,0.006060606,-0.032967033,0,0,1.797424e-04,-1.236736e-04,0.00000000,0.00000000
249234235,202106,1,0,0,0.5555556,0.05820106,0.0045558310,0.003779038,0.0057654980,⋯,2.823259e-03,0.0030559121,0.054545455,0.030674847,0,0,1.910595e-03,1.759432e-03,0.00000000,0.00000000
249244449,202106,1,0,0,0.6262626,0.78042328,0.0018472682,0.013455802,0.0055297679,⋯,5.060762e-05,-0.0027974136,0.000000000,-0.010989011,0,0,3.794561e-04,-1.780900e-03,0.00000000,0.00000000
249244739,202106,1,0,0,0.6666667,0.85978836,0.0063113650,0.010056512,0.0083585534,⋯,2.777995e-03,-0.0042511625,0.000000000,0.000000000,0,0,2.196851e-04,-5.194291e-04,0.00000000,-0.20454545
249255456,202106,1,0,0,0.6565657,0.85978836,0.0076678816,0.023735786,0.0018596944,⋯,-7.567064e-03,-0.0196198393,-0.012500000,-0.087912088,0,0,-2.682258e-03,-4.007025e-03,0.00000000,0.00000000
249255747,202106,1,0,0,0.5656566,0.23809524,0.0035065101,0.010545742,0.0025843883,⋯,3.066794e-02,0.0496055021,0.042424242,0.006134969,0,0,1.917252e-02,2.704171e-02,0.00000000,0.00000000
249257429,202106,1,0,0,0.6262626,0.85978836,0.0048605724,0.011451997,0.0092136410,⋯,2.994650e-02,0.0329394668,0.103030303,0.098159509,0,0,1.403988e-02,-1.595389e-03,0.00000000,0.00000000
249273979,202106,1,0,0,0.6060606,1.00000000,0.0024374799,0.012548706,0.0061468254,⋯,1.383595e-02,0.0113424610,0.078787879,0.036809816,0,0,-9.307520e-05,-2.720819e-04,0.00000000,0.00000000


In [88]:
# inicilizo el dataset  drealidad
drealidad <- realidad_inicializar( dfuture, PARAM)

#### Tabla Prediccion

In [89]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

In [90]:
PARAM$cortes

[1]  6000  6500  7000  7500  8000  8500  9000  9500 10000 10500 11000 11500
[13] 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500
[25] 18000 18500 19000

In [91]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  res <- realidad_evaluar( drealidad, tb_prediccion)

  options(scipen = 999)
  cat( "Envios=", envios, "\t",
    " TOTAL=", res$total,
    "  Public=", res$public,
    " Private=", res$private,
    "\n",
    sep= ""
  )

}

Envios=6000	 TOTAL=-120000000  Public=-122733333 Private=-118828571
Envios=6500	 TOTAL=-130000000  Public=-132666667 Private=-128857143
Envios=7000	 TOTAL=-140000000  Public=-142866667 Private=-138771429
Envios=7500	 TOTAL=-150000000  Public=-152400000 Private=-148971429
Envios=8000	 TOTAL=-160000000  Public=-162866667 Private=-158771429
Envios=8500	 TOTAL=-170000000  Public=-173000000 Private=-168714286
Envios=9000	 TOTAL=-180000000  Public=-182800000 Private=-178800000
Envios=9500	 TOTAL=-190000000  Public=-194066667 Private=-188257143
Envios=10000	 TOTAL=-200000000  Public=-204800000 Private=-197942857
Envios=10500	 TOTAL=-210000000  Public=-213933333 Private=-208314286
Envios=11000	 TOTAL=-220000000  Public=-223733333 Private=-218400000
Envios=11500	 TOTAL=-230000000  Public=-232733333 Private=-228828571
Envios=12000	 TOTAL=-240000000  Public=-241800000 Private=-239228571
Envios=12500	 TOTAL=-250000000  Public=-251400000 Private=-249400000
Envios=13000	 TOTAL=-260000000  Public=-26

In [92]:
write_yaml( PARAM, file="PARAM.yml")

In [93]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "vie. oct. 10 11:58:04 2025"

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar04**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla